In [173]:
# 
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")


from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report, confusion_matrix , auc,roc_curve
from sklearn.model_selection import GridSearchCV 
from sklearn.inspection import permutation_importance

In [2]:
def ema(values):
    ewm=0
    for i in values:
        ewm=(ewm+i)/2
    return ewm

In [206]:
# data
# if changed order, update the predict.py
col_list=["LB","UB","ObjCoeff","yLP","Dual","RC","yInt"]#"LB", "ObjCoeff","Dual","RC","y_lp_sum","ylp","yint"]#"RC", "ylp", "yint"]#,"Dual","y_lp_sum","LB", "Routing_Cost","Capacity","Fixed_Cost"]
df = pd.read_csv("data/{}.csv".format("noRootLifter"))[col_list]#allSolsWithRC probSpecificScaling

# feature eng
t=5
df["ylp_std"]=0
df["ylp_ema"]=0
df["y_0"]=0
df["y_1"]=0

df["dual_std"]=0
df["dual_ema"]=0
df["dual_0"]=0

df["rc_std"]=0

# df["gap"]=df.UB-df.LB 
df["rc_0"]=0
 
for idx,row in df.iterrows(): 
    yRow=np.float_( row.yLP.split("_")[t:-1])
    dRow=np.float_( row.Dual.split("_")[t:-1])
    rcRow=np.float_( row.RC.split("_")[t:-1])
    df.loc[idx,"yLP"] = yRow.mean()#[-1]
    df.loc[idx,"Dual"] = dRow.mean()
    df.loc[idx,"RC"] = rcRow.mean() 
    
    df.loc[idx,"ylp_ema"] = ema(yRow)
    df.loc[idx,"dual_ema"] = ema(dRow)
    
#     df.loc[idx,"ylp_med"] = np.median(yRow)
#     df.loc[idx,"dual_med"] = np.median(dRow)
#     df.loc[idx,"rc_med"] = np.median(rcRow)
    df.loc[idx,"ylp_std"] = yRow.std()
    df.loc[idx,"dual_std"] = dRow.std()
    df.loc[idx,"rc_std"] = rcRow.std()
    
    #number of time the daul has been zero
    df.loc[idx,"dual_0"]=len([i for i in dRow if abs(i)<0.1])/len(dRow)
    df.loc[idx,"rc_0"]=len([i for i in rcRow if (i)<0])/len(rcRow)
    df.loc[idx,"y_0"]=len([i for i in yRow if abs(i)<=0.1])/len(yRow)
    df.loc[idx,"y_1"]=len([i for i in yRow if abs(i)>=0.9])/len(yRow)

# df["x"]=0
# df["rc_sig"]=df.UB-df.LB+df.RC
df.loc["gap"]=(df.UB-df.LB )/(1+df.RC )
df[col_list]=df[col_list].astype("float")
df["yInt"]=df["yInt"].fillna(0).astype("int")

df.dropna(inplace=True)
df.drop(["LB","UB"],axis="columns",inplace=True)
print(df.shape)
print(df.yInt.value_counts()) 
print(df.corr().yInt.sort_values())
print(abs(df.yLP-df.yInt).sum())
df.head(2)
# df.describe()

(1440, 14)
0    789
1    651
Name: yInt, dtype: int64
y_0        -0.793001
RC         -0.505182
dual_0     -0.464039
Dual       -0.401842
dual_ema   -0.354114
ObjCoeff   -0.272829
rc_std      0.077641
ylp_std     0.244245
dual_std    0.291710
rc_0        0.635045
y_1         0.671350
ylp_ema     0.808613
yLP         0.814403
yInt        1.000000
Name: yInt, dtype: float64
216.08881344975134


,ObjCoeff,yLP,Dual,RC,yInt,ylp_std,ylp_ema,y_0,y_1,dual_std,dual_ema,dual_0,rc_std,rc_0
0,1.000000,0.002018,0.0,478.406331,0,0.004196,0.000002,1.0,0.0,0.0,0.0,1.0,957.2373,0.0
1,0.329296,0.002018,0.0,0.000000,0,0.004196,0.000002,1.0,0.0,0.0,0.0,1.0,0.0000,0.0


In [207]:
def QuntileScore(df):
    df["pred"]=-1
    df.loc[df.score>=df.score.quantile(0.75),"pred"]=1
    df.loc[df.score<=df.score.quantile(0.25),"pred"]=0
    print(len(df[df.pred!=-1])/len(df),1-abs(df[df.pred!=-1].yInt-df[df.pred!=-1].pred).sum()/len(1e-10+df[df.pred!=-1]))

In [236]:
# split and scale
# df_=pd.concat([df,df[df.yInt==1].head(int(0.28*len(df)))])

def Split():
    train, test = train_test_split(pd.concat([df,df[df.yInt==1].head(
        int(len(df[df.yInt==0]) -len(df[df.yInt==1]))
    )]), test_size=0.25,random_state=42)
    # print(100*train.yInt.value_counts(normalize=True))
    print(100*test.yInt.value_counts(normalize=True))
    train, val = train_test_split(train, test_size=0.005,random_state=42)
    print(100*train.yInt.value_counts(normalize=True))
    print(100*val.yInt.value_counts(normalize=True))

    # ech problem should be scaled separately
    for col in df.columns:
    #     if  col=="yint" or col=="ylp" or col=="y_lp_sum" or col=="y_0" or col=="y_1" or col=="ylp_ema" or col=="ylp_med":
    #         continue
        if col=="gap" or col=="RC"or col=="rc_med" or col=="rc_std" or col=="rc_ema" or col=="Dual" or col=="dual_med" or col=="dual_std" or col=="dual_ema" or col=="d_c":
            scaler=StandardScaler().fit(train[[col]])
            train[col]=scaler.transform(train[[col]])
            test[col]=scaler.transform(test[[col]])
            pickle.dump(scaler, open("scaler_{}.sav".format(col),'wb'))
    return train,test,val

In [237]:
# boost
train,test,val=Split()
trainScores=[]
testScores=[] 
clf = GradientBoostingClassifier(random_state=42
#                                 ,learning_rate=0.01
#                                  ,n_estimators=250
                                 ,max_depth=4
#                                  ,min_samples_split=0.9
#                                  ,subsample=0.7
#                                  ,learning_rate= 0.025, max_depth= 30, n_estimators= 1000
                                ).fit(train.drop("yInt",axis="columns"), train.yInt)
trainScore=100*clf.score(train.drop("yInt",axis="columns"), train.yInt)
testScore=100*clf.score(test.drop("yInt",axis="columns"), test.yInt)
print("Score",trainScore,testScore)



fpr_gb, tpr_gb, _ = roc_curve(test.yInt, clf.decision_function(test.drop("yInt",axis="columns")))
fpr_gb_, tpr_gb_, _ = roc_curve(train.yInt, clf.decision_function(train.drop("yInt",axis="columns")))
fpr_gb__, tpr_gb__, _ = roc_curve(df.yInt, clf.decision_function(df.drop("yInt",axis="columns")))
print("AUC",auc(fpr_gb_, tpr_gb_),auc(fpr_gb, tpr_gb), "=>",auc(fpr_gb__, tpr_gb__))

train["score"]=clf.predict_proba(train.drop("yInt",axis="columns"))[:,1]
QuntileScore(train)
test["score"]=clf.predict_proba(test.drop("yInt",axis="columns"))[:,1]
QuntileScore(test) 

# pickle.dump(clf, open("gboost.sav", 'wb'))
# r = permutation_importance(clf, df.drop("yInt",axis="columns"), df.yInt
#                            ,n_repeats=100
# #                            , n_jobs=-1
#                            ,random_state=42
#                           )
# for i in r.importances_mean.argsort()[::-1]:
#     print(f"{df.drop('yInt',axis='columns').columns[i]:<8}" f" {r.importances_mean[i]:.3f}" 
#           f" +/- {r.importances_std[i]:.3f}" f" {r.importances_mean[i] - 2 * r.importances_std[i] > 0}"
#          )

1    60.803059
0    39.196941
Name: yInt, dtype: float64
1    62.884615
0    37.115385
Name: yInt, dtype: float64
0    62.5
1    37.5
Name: yInt, dtype: float64
Score 99.16666666666667 95.41108986615679
AUC 0.9995088019521161 0.9864396379812854 => 0.9790640897595392
0.5 1.0
0.5009560229445507 0.9961832061068703


In [149]:
# mlp
mlp = MLPClassifier(
#     hidden_layer_sizes=(5000, 500)
    random_state=1,max_iter=100000
    , tol=1e-27
).fit(    train.drop("yInt",axis="columns"), train.yInt)
# solver='lbfgs', alpha=1e-5,  hidden_layer_sizes=(50, 20), random_state=1,max_iter=1000
print(100*mlp.score(train.drop("yInt",axis="columns"), train.yInt))
print(100*mlp.score(test.drop("yInt",axis="columns"), test.yInt))
pickle.dump(mlp, open("mlp.sav", 'wb'))


r = permutation_importance(mlp, test.drop("yInt",axis="columns"), test.yInt, n_repeats=100, random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    print( f"{test.columns[i]:<8}" f" {r.importances_mean[i]:.3f}" f" +/- {r.importances_std[i]:.3f}" f" {r.importances_mean[i] - 2 * r.importances_std[i] > 0}")

mlp
# 0.9564102564102565
# 0.9387755102040817
# ylp      0.349 +/- 0.046 True
# RC       0.055 +/- 0.022 True


93.87085005472456
93.79760609357997
RC       0.053 +/- 0.007 True
ylp_std  0.051 +/- 0.008 True
yLP      0.019 +/- 0.006 True
rc_std   0.009 +/- 0.005 False
dual_0   0.005 +/- 0.003 False
ylp_ema  0.004 +/- 0.004 False
y_0      0.004 +/- 0.003 False
y_1      0.003 +/- 0.003 False
ObjCoeff 0.002 +/- 0.002 False
dual_std 0.002 +/- 0.003 False
dual_ema 0.001 +/- 0.002 False
Dual     -0.001 +/- 0.002 False
yInt     -0.002 +/- 0.002 False


MLPClassifier(max_iter=100000, random_state=1, tol=1e-27)

In [150]:
# logistic
logstic = LogisticRegression(max_iter=10000, tol=1e-10)#make_pipeline(StandardScaler(), LogisticRegression())
logstic.fit(train.drop("yInt",axis="columns"), train.yInt) 

print(logstic.score(train.drop("yInt",axis="columns"), train.yInt))
print(logstic.score(test.drop("yInt",axis="columns"), test.yInt))
pickle.dump(logstic, open("logistic.sav", 'wb'))

r = permutation_importance(logstic, test.drop("yInt",axis="columns"), test.yInt, n_repeats=100, random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    print( f"{test.columns[i]:<8}" f" {r.importances_mean[i]:.3f}" f" +/- {r.importances_std[i]:.3f}" f" {r.importances_mean[i] - 2 * r.importances_std[i] > 0}")
importance = logstic.coef_[0]
for i,v in enumerate(importance):
    print('Feature: {}, Score: {}'.format (test.columns[i],v))
    # 0.9538461538461539
# 0.9387755102040817

# 0.9140625
# 0.9166666666666666

# 0.9157986111111112
# 0.9131944444444444


0.9354250273622765
0.9445048966267682
ylp_std  0.042 +/- 0.006 True
RC       0.031 +/- 0.006 True
ylp_ema  0.016 +/- 0.004 True
yLP      0.013 +/- 0.005 True
y_0      0.006 +/- 0.003 True
dual_ema 0.003 +/- 0.002 False
ObjCoeff 0.002 +/- 0.002 False
y_1      0.002 +/- 0.003 False
dual_std 0.002 +/- 0.001 False
rc_std   0.002 +/- 0.001 False
Dual     0.001 +/- 0.001 False
dual_0   0.000 +/- 0.000 False
yInt     0.000 +/- 0.000 False
Feature: ObjCoeff, Score: -0.8756462189364983
Feature: yLP, Score: 1.8175542942172593
Feature: Dual, Score: 0.07044974955302002
Feature: RC, Score: -1.7981189873471957
Feature: yInt, Score: 0.13341485770808634
Feature: ylp_std, Score: 2.4920701474674116
Feature: ylp_ema, Score: -1.9700185333096643
Feature: y_0, Score: 1.0727496438154067
Feature: y_1, Score: 0.5189935188374641
Feature: dual_std, Score: 0.1647333254734069
Feature: dual_ema, Score: -0.37249080665560647
Feature: dual_0, Score: 0.011240154914805317
Feature: rc_std, Score: 0.5366643571640523


In [151]:
# svm
svm_ = svm.SVC()#make_pipeline(StandardScaler(), svm.SVC())
svm_.fit(train.drop("yInt",axis="columns"), train.yInt) 

print(svm_.score(train.drop("yInt",axis="columns"), train.yInt))
print(svm_.score(test.drop("yInt",axis="columns"), test.yInt))
pickle.dump(svm_, open("svm.sav", 'wb'))

r = permutation_importance(svm_, test.drop("yInt",axis="columns"), test.yInt, n_repeats=100, random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    print( f"{test.columns[i]:<8}" f" {r.importances_mean[i]:.3f}" f" +/- {r.importances_std[i]:.3f}" f" {r.importances_mean[i] - 2 * r.importances_std[i] > 0}")
# 0.9560117302052786
# 0.9523809523809523

# 0.9262152777777778
# 0.9097222222222222

# 0.9236111111111112
# 0.9131944444444444


0.9332360452389639
0.9423286180631121


KeyboardInterrupt: 

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50, 20),(500, 20), (50,200),(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05, 1e-5, 0.1],
    'learning_rate': ['constant','adaptive'],
}
mlpcv = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
mlpcv.fit(train.drop("yint",axis="columns"), train.yint)
# Best paramete set
print('Best parameters found:\n', mlpcv.best_params_)
# All results
means = mlpcv.cv_results_['mean_test_score']
stds = mlpcv.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, mlpcv.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
df_=pd.read_csv("ml_output.csv")[["Reduced_Cost","y_lp"]]
print(clf.score(df_, df_.y_lp))

In [ ]:
plt.figure(figsize=(250,10))
plt.scatter(range(len(test["y"])),test["y"])
plt.scatter(range(len(test["y"])),test["y_int"])

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(train.drop("y_int",axis="columns"), train.y_int) 

In [ ]:
# test["score"]=pipe.decision_function(test.drop(["y_int"],axis="columns"))
# test["pred"]=pipe.predict(test.drop(["score","y_int"],axis="columns"))
# print(100*len(test[test.y!=test.y_int])/len(test))
# print(test[test.y!=test.y_int][["y_int","y"]].sort_values("y").head(1000))
print(test[["y_lp","y_int","score","pred"]].describe())
test[["y_lp","y_int","score","pred"]].sort_values("score")[test.pred!=test.y_int]

In [ ]:
# boost
trainScores=[]
testScores=[]
for d in range(1000):#range(7,30):
    d=np.random.randint(low=1,high=50)
    t=np.random.randint(1,10000)
    print(d,t)
    clf = GradientBoostingClassifier(random_state=42,
#                                 learning_rate=0.01,
                                 n_estimators=t
                                 ,max_depth=d
                                ).fit(train.drop("yInt",axis="columns"), train.yInt)
    trainScore=clf.score(train.drop("yInt",axis="columns"), train.yInt)
    testScore=clf.score(val.drop("yInt",axis="columns"), val.yInt)
    print(trainScore,testScore)
    trainScores.append(trainScore)
    testScores.append(testScore)
#     if trainScore>0.999:
#         for t in range(100,100000,250):
#             clf = GradientBoostingClassifier(random_state=42,
#         #                                 learning_rate=0.01,
#                                          n_estimators=t,
#                                          max_depth=d
#                                         ).fit(train.drop("yInt",axis="columns"), train.yInt)
#             trainScore=clf.score(train.drop("yInt",axis="columns"), train.yInt)
#             testScore=clf.score(val.drop("yInt",axis="columns"), val.yInt)
#             print(d,t,trainScore,testScore)
#             trainScores.append(trainScore)
#             testScores.append(testScore)
#             if testScore>0.99:
#                 break
#         break

# pickle.dump(clf, open("gboost.sav", 'wb'))

# r = permutation_importance(clf, test.drop("yint",axis="columns"), test.yint, n_repeats=100, random_state=0)
# for i in r.importances_mean.argsort()[::-1]:
#     print( f"{test.columns[i]:<8}" f" {r.importances_mean[i]:.3f}" f" +/- {r.importances_std[i]:.3f}" f" {r.importances_mean[i] - 2 * r.importances_std[i] > 0}")


# 0.9208416833667334
# 0.9151636606546426
# ylp      0.392 +/- 0.008 True
# y_lp_sum 0.086 +/- 0.005 True
# ObjCoeff 0.011 +/- 0.002 True
# RC       0.004 +/- 0.002 False
# Dual     0.002 +/- 0.002 False

In [ ]:
parameters = {
#     "loss":["deviance"],
    "learning_rate": [0.001,0.01, 0.025, 0.05, 0.075, 0.1],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,7,18, 30],
#     "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10, 200, 500, 1000, 3000, 5000, 10000]
    }
clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=3, n_jobs=-1,verbose=100)

clf.fit(df.drop("yInt",axis="columns"), df.yInt)
print(clf.score(df.drop("yInt",axis="columns"), df.yInt))
print(clf.best_params_)
